In [1]:
#Importing ML Libraries
import math
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html 
from dash.dependencies import Input,Output

C:\Users\k.park99\AppData\Local\Temp\ipykernel_19280\3957145141.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\k.park99\AppData\Local\Temp\ipykernel_19280\3957145141.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#ML Libraries
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.stats.mstats import trimmed_var
from sklearn.metrics import silhouette_score

In [3]:
#Reading Data into DataFrame
data=pd.read_csv(r'C:\Users\k.park99\Downloads\Fingertips Data Science Pro\Python\marketing_campaign.csv',sep='\t')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [4]:
#Data Cleaning 
data.dropna(inplace=True)
data.drop(['ID','Dt_Customer'],axis=1,inplace=True)
data

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,7,0,0,0,0,0,0,3,11,1
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,5,0,0,0,0,0,0,3,11,0
2,1965,Graduation,Together,71613.0,0,0,26,426,49,127,...,4,0,0,0,0,0,0,3,11,0
3,1984,Graduation,Together,26646.0,1,0,26,11,4,20,...,6,0,0,0,0,0,0,3,11,0
4,1981,PhD,Married,58293.0,1,0,94,173,43,118,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1967,Graduation,Married,61223.0,0,1,46,709,43,182,...,5,0,0,0,0,0,0,3,11,0
2236,1946,PhD,Together,64014.0,2,1,56,406,0,30,...,7,0,0,0,1,0,0,3,11,0
2237,1981,Graduation,Divorced,56981.0,0,0,91,908,48,217,...,6,0,1,0,0,0,0,3,11,0
2238,1956,Master,Together,69245.0,0,1,8,428,30,214,...,3,0,0,0,0,0,0,3,11,0


In [5]:
#Feature Extraction 
data['age']=2022-data['Year_Birth']
data.drop(['Year_Birth'],axis=1,inplace=True)

In [6]:
#Feature Encoding
data.Education.unique()
data.Education.replace({'Basic':1,'Graduation':2,'2n Cycle':3,'Master':3,'PhD':4},inplace=True)

In [7]:
#Feature Extraction 2
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]
data.drop(['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds'],axis=1,inplace=True)

In [8]:
#Feature Extraction 3
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})
data["Children"]=data["Kidhome"]+data["Teenhome"]
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]
data.drop(['Marital_Status','Kidhome','Teenhome','Children'],axis=1,inplace=True)

In [9]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 0 to 2239
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Education            2216 non-null   int64  
 1   Income               2216 non-null   float64
 2   Recency              2216 non-null   int64  
 3   NumDealsPurchases    2216 non-null   int64  
 4   NumWebPurchases      2216 non-null   int64  
 5   NumCatalogPurchases  2216 non-null   int64  
 6   NumStorePurchases    2216 non-null   int64  
 7   NumWebVisitsMonth    2216 non-null   int64  
 8   AcceptedCmp3         2216 non-null   int64  
 9   AcceptedCmp4         2216 non-null   int64  
 10  AcceptedCmp5         2216 non-null   int64  
 11  AcceptedCmp1         2216 non-null   int64  
 12  AcceptedCmp2         2216 non-null   int64  
 13  Complain             2216 non-null   int64  
 14  Z_CostContact        2216 non-null   int64  
 15  Z_Revenue            2216 non-null   i

In [10]:
#Further Data Cleaning 
data.drop(['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain'
           ,'Response'],axis=1,inplace=True)

In [11]:
#Further Data Cleaning
data.drop(['Living_With'],axis=1,inplace=True)

In [12]:
data.head()

,Education,Income,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Z_CostContact,Z_Revenue,age,Spent,Family_Size
0,2,58138.0,58,3,8,10,4,7,3,11,65,1617,1
1,2,46344.0,38,2,1,1,2,5,3,11,68,27,3
2,2,71613.0,26,1,8,2,10,4,3,11,57,776,2
3,2,26646.0,26,2,2,0,4,6,3,11,38,53,3
4,4,58293.0,94,5,5,3,6,5,3,11,41,422,3


In [69]:
app=dash.Dash()

In [70]:
app.layout = html.Div(
    style={'textAlign': 'center', 'width': '800px', 'font-family': 'Verdana'},
    children=[
        html.H1('Clustering Analysis'),
        html.H2('High Variance Features'),
        dcc.Graph(id='bar-chart'),
        html.H3('Variance Options'),
        dcc.RadioItems(
            options=[
                {'label': 'trimmed', 'value': True},
                {'label': 'not trimmed', 'value': False}
            ],
            value=True,
            id='trim-button'
        ),
        
        html.H2('Dimensionality Reduction'),
        html.H3('Number of Clusters'),
        dcc.Slider(min=2, max=13, step=1, value=2, id='pca-slider'),
        html.Div(id='metrics'),
        dcc.Graph(id='scatter-plot'),
        
        html.H2('Prediction'),
        html.H3('Number of Clusters'),
        dcc.Slider(min=2, max=13, step=1, value=2, id='n-slider'),
        html.H3('Education Level'),
        dcc.RadioItems(
            options=[
                {'label': 'Basic', 'value': 1},
                {'label': 'Graduation', 'value': 2},
                {'label': 'Master', 'value': 3},
                {'label': 'PhD', 'value': 4}
            ],
            value=2,
            id='education-button'
        ),
        html.H3('Income'),
        dcc.Slider(min=1730, max=666666, value=4000, id='income-slider'),
        html.H3('Age'),
        dcc.Slider(min=26, max=129, value=26, id='age-slider'),
        html.H3('Family Size'),
        dcc.Slider(min=1, max=5, step=1, value=2, id='family-slider'),
        html.Div(id='label')
    ]
)



In [71]:
@app.callback(
    Output('bar-chart', 'figure'),
    Input('trim-button', 'value')
)
def serve_bar_chart(trimmed=True):
    model1=StandardScaler()
    data1=model1.fit_transform(data)
    data1=pd.DataFrame(data1,columns=data.columns)
    if trimmed:
        fig = px.bar(x=data1.apply(trimmed_var).sort_values().tail(5).index,
                     y=data1.apply(trimmed_var).sort_values().tail(5))
        fig.update_layout(xaxis_title='Feature Names', yaxis_title='Variance')
    else:
        fig = px.bar(x=data1.var().sort_values().tail(5).index,
                     y=data1.var().sort_values().tail(5))
        fig.update_layout(xaxis_title='Feature Names', yaxis_title='Variance')
    return fig



In [72]:
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('pca-slider', 'value')
)
def pca_func(n=2):
    model1 = PCA(n_components=3)
    X_pca = model1.fit_transform(data)
    X_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2','PC3'])
    model2=KMeans(n_clusters=n)
    model2.fit(data)
    X_pca['labels'] = model2.labels_
    fig=px.scatter_3d(X_pca,'PC1','PC2','PC3',color='labels')
    return fig

In [73]:
@app.callback(
    Output('metrics', 'children'),
    Input('pca-slider', 'value')
)
def serve_metrics(n=2):
    model=KMeans(n_clusters=n)
    model.fit(data)
    text=[
        html.H3(f"Inertia: {model.inertia_}"),
        html.H3(f"silhouette Score: {silhouette_score(data,model.labels_)}")   
    ]
    return text

In [74]:
@app.callback(
    Output('label', 'children'),
    Input('education-button', 'value'),
    Input('income-slider', 'value'),
    Input('age-slider', 'value'),
    Input('family-slider', 'value'),
    Input('n-slider', 'value'),
    
)
def update_prediction(X1=2,X2=4000,X3=26,X4=2,n=2):
    model=KMeans(n_clusters=n)
    model.fit(data)
    input_X=np.array([X1,X2,data['Recency'].mean(),data['NumDealsPurchases'].mean(),data['NumWebPurchases'].mean(),
                      data['NumCatalogPurchases'].mean(),data['NumStorePurchases'].mean(),
                      data['NumWebVisitsMonth'].mean(),data['Z_CostContact'].mean(),data['Z_Revenue'].mean(),
                      X3,data['Spent'].mean(),X4]).reshape(1,-1)
    prediction=model.predict(input_X)[0]
    text=[html.H3(f'Cluster Label: {prediction}')]
    return text

In [75]:
if __name__ == '__main__':
    app.run(jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/


C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names

C:\Users\k.park99\anaconda3\lib\site-pac